### The `__del__` Method

In [ ]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [2]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'Person({self.name})'

    def __del__(self):
        print('__del__ called for {self}...')

In [3]:
p = Person('Alex')

In [4]:
id_p = id(p)
ref_count(id_p)

1

In [5]:
p = None

__del__ called for {self}...


In [6]:
p = Person('Alex')

In [7]:
del p

__del__ called for {self}...


In [8]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'Person({self.name})'

    def __del__(self):
        print('__del__ called for {self}...')

    def gen_ex(self):
        raise ValueError('Something went bump...')

In [9]:
p = Person('Eric')
p_id = id(p)
ref_count(p_id)

1

In [12]:
try:
    p.gen_ex()
except ValueError as ex:
    error = ex
    print(ex)

Something went bump...


In [11]:
ex

NameError: name 'ex' is not defined

In [13]:
error

ValueError('Something went bump...')

In [14]:
ref_count(p_id)

2

In [15]:
dir(error.__traceback__)

['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next']

In [16]:
dir(error.__traceback__.tb_frame)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace',
 'f_trace_lines',
 'f_trace_opcodes']

In [17]:
type(error.__traceback__.tb_frame.f_locals)

dict

In [21]:
for key, value in error.__traceback__.tb_frame.f_locals.copy().items():
    if isinstance(value, Person):
        print(key, value, id(value))

p Person(Eric) 139680368253584


In [22]:
ref_count(p_id)

2

In [23]:
p_id

139680368253584

In [24]:
del p

In [25]:
del error

In [26]:
ref_count(p_id)

1

In [27]:
class Person:
    def __del__(self):
        raise ValueError('Seomthing went bump...')

In [29]:
p = Person()

In [30]:
del p
print('All good, no exceptions interrupted our code.')

Exception ignored in: <function Person.__del__ at 0x7f09ddca1120>
Traceback (most recent call last):
  File "/tmp/ipykernel_42382/3693585920.py", line 3, in __del__
ValueError: Seomthing went bump...


All good, no exceptions interrupted our code.


In [31]:
import sys

In [33]:
sys.stderr, sys.stdout

(<ipykernel.iostream.OutStream at 0x7f0a00bb4790>,
 <ipykernel.iostream.OutStream at 0x7f0a00bb44c0>)

In [35]:
class ErrToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stderr = sys.stderr

    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stderr = self._file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stderr = self._current_stderr
        if self._file:
            self._file.close()
        return False

In [38]:
p = Person()

In [39]:
with ErrToFile('err.txt'):
    del p
    print(100)

print(200)
print(300)

100
200
300


In [42]:
with open('err.txt') as f:
    print(f.readlines())

['Exception ignored in: <function Person.__del__ at 0x7f09ddca1120>\n', 'Traceback (most recent call last):\n', '  File "/tmp/ipykernel_42382/3693585920.py", line 3, in __del__\n', 'ValueError: Seomthing went bump...\n']
